# Annotating pivots

Using the pigeon library.

In [2]:
import takco
tables = list(takco.TableSet.load('../../output/tabel-10k-2/0-load/*.jsonl'))
headers = {takco.reshape.table_get_headerId(t):t.get('tableHeaders') for t in tables}
len(headers)

4929

In [25]:
def show_pivot(pivot):
    tableHeaders = headers[pivot['headerId']]
    def make_cell(c,ri,ci,pivot):
        cell = c.get("tdHtmlString", "<th>"+c.get('text','')+"</th>")
        if ri == pivot['level'] and ci in range(pivot['colfrom'],pivot['colto']+1):
            cell = cell[:3] + ' style="background:#8c8;"' + cell[4:]
        return cell
    head = [
        [make_cell(c,ri,ci,pivot) for ci,c in enumerate(r)]
        for ri,r in enumerate(tableHeaders)
    ]
    head = [[c.replace("span=", "=") for c in row] for row in head]
    head = "".join(f'<tr>{"".join(row)}</tr>' for row in head)
    html = "<table>" + head + "</table>"
    display.display(display.HTML( html ))
    

import itertools
ns = takco.reshape.NumSuffix()
pivots = takco.reshape.yield_pivots(headers.values(), {'numsuffix':ns})
pivots = itertools.islice(pivots, 50)

from pigeon import annotate
annotations = annotate(
    list(pivots),
    options=['good', 'bad'],
    display_fn = show_pivot
)

HTML(value='0 examples annotated, 51 examples left')

Output()

Annotation done.


In [26]:
annotations

[({'headerId': 7216962826754076956,
   'level': 0,
   'colfrom': 0,
   'colto': 0,
   'heuristic': 'numsuffix'},
  'bad'),
 ({'headerId': 3441036834358993182,
   'level': 0,
   'colfrom': 0,
   'colto': 0,
   'heuristic': 'numsuffix'},
  'bad'),
 ({'headerId': 3836035899065722383,
   'level': 0,
   'colfrom': 0,
   'colto': 0,
   'heuristic': 'numsuffix'},
  'good'),
 ({'headerId': 778735725590271279,
   'level': 0,
   'colfrom': 0,
   'colto': 0,
   'heuristic': 'numsuffix'},
  'bad'),
 ({'headerId': 2036622794261251368,
   'level': 0,
   'colfrom': 0,
   'colto': 0,
   'heuristic': 'numsuffix'},
  'good'),
 ({'headerId': 2873865634436721714,
   'level': 0,
   'colfrom': 4,
   'colto': 14,
   'heuristic': 'numsuffix'},
  'good'),
 ({'headerId': 130443667544424743,
   'level': 0,
   'colfrom': 1,
   'colto': 2,
   'heuristic': 'numsuffix'},
  'good'),
 ({'headerId': 5596464666231700013,
   'level': 0,
   'colfrom': 1,
   'colto': 1,
   'heuristic': 'numsuffix'},
  'bad'),
 ({'headerId'